# VizieR Queries

In [3]:
from astroquery.vizier import Vizier
import numpy as np
import pandas as pd

#### Cantat-Gaudin Catalog

Data from <u>**Painting a portrait of the Galactic disc with its stellar clusters**</u> by Cantat-Gaudin et al, 2020.<a href= "https://ui.adsabs.harvard.edu/abs/2020A%26A...640A...1C/abstract" > [Link to the Paper]</a>

The data is available in two different tables, one listing the cluster parameters, the other one listing the members of each cluster. By using `astroquery.vizier` we can query the VizieR data bank with ease and retrieve the tables we want to use. 

In [8]:
%%time  
# C A N T A T - G A U D I N 
# shows time spent bz the CPU processing the task, wall shows full elapsed time 

# Pulling the catalogs from the Virtual Observatory
CATALOGUE = "J/A+A/640/A1"
cols_cantat = ['Cluster', 'GaiaDR2','nbstars07', 'RA_ICRS', 'DE_ICRS', 'r50', 'pmRA', 'pmRA_', 'pmDE', 'plx', 'DistPc', 'AgeNN', 'AVNN', 'Gmag', 'BP-RP', 'proba', 'GLON', 'GLAT']
data_cantat_vizier = Vizier(catalog= CATALOGUE, row_limit=-1, columns=cols_cantat).query_constraints()
data_cantat_vizier

CPU times: user 2.63 s, sys: 423 ms, total: 3.05 s
Wall time: 15.3 s


TableList with 2 tables:
	'0:J/A+A/640/A1/table1' with 12 column(s) and 2017 row(s) 
	'1:J/A+A/640/A1/nodup' with 10 column(s) and 234128 row(s) 

In [ ]:
# Splitting the two lists, astropy tables
cluster_cantat_list = data_cantat_vizier[0]
member_cantat_list = data_cantat_vizier[1]

# Converting to pandas DataFrames for easier handling 
clusters_cantat = cluster_cantat_list.to_pandas()
members_cantat = member_cantat_list.to_pandas()

### Gaia - Apogee Crossmatch

In [17]:
def query_gaia(IDs, cols = None):
    if cols is None:
        cols = ['Source','RA_ICRS', 'DE_ICRS', 'e_RA_ICRS', 'e_DE_ICRS', 'Gmag', 
                'Plx', 'e_Plx', 'PM', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE','RV', 'e_RV',
                'RADEcor', 'RAPlxcor', 'DEPlxcor', 'RApmRAcor', 'RApmDEcor', 'PlxpmRAcor', 'DEpmRAcor', 'DEpmDEcor', 'PlxpmDEcor', 'pmRApmDEcor']
    
    CATALOGUE = 'I/355/gaiadr3'
    data_vizier = Vizier(catalog= CATALOGUE, row_limit=-1, columns=cols).query_constraints(Source=IDs)
    df_gaia = data_vizier[0].to_pandas()
    print(data_vizier)
    return df_gaia

def query_apogee(IDS, cols = None):
    if cols is None:
        cols = ['APOGEE_ID', 'GaiaEDR3', 'RAJ2000', 'DEJ2000', 'RV', 'e_RV']
    
    CATALOGUE_APOGEE = 'III/286/catalog'
    data_apogee = Vizier(catalog=CATALOGUE_APOGEE, columns=cols).query_constraints(GaiaEDR3=IDS)
    df_apogee = data_apogee[0].to_pandas()
    return df_apogee

In [18]:
# Example: Collinder 135
collinder135_sources = members_cantat.loc[members_cantat['Cluster'] == 'Collinder_135']
IDS_collinder = collinder135_sources['GaiaDR2'].astype(str).tolist()

df_gaia_mG = query_gaia(IDS_collinder)
df_apogee_mG = query_apogee(IDS_collinder)
df_apogee_mG.count()

TableList with 1 tables:
	'0:I/355/gaiadr3' with 25 column(s) and 320 row(s) 


APOGEE      2
GaiaEDR3    2
RAJ2000     2
DEJ2000     2
RV          0
e_RV        0
dtype: int64